# Web Scraping and time series

In [29]:
#import the necessary packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import Request, urlopen
import regex as re
import matplotlib as plt
from bs4 import BeautifulSoup as bsoup
%matplotlib inline

### go to the Eurosport Paris Roubaix cycling results page and load it into a pandas dataframe

https://www.eurosport.com/cycling/paris-roubaix/2022/live-road-race-men_mtc1323920/live.shtml

Hint, pandas read_html() will fail to find the table even though it is on the page.  You need to find the table (class='standing-table') with a selenium driver.get() or BeautifulSoup, then pass that to pandas read_html().

In [30]:
# Reading in the web page Then displaying

# Eurosport Paris Roabix cycling results
url = "https://www.eurosport.com/cycling/paris-roubaix/2022/live-road-race-men_mtc1323920/live.shtml"

# Make the request and open the url :
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Create a beutiful soup object so we can work with the data
soup = bsoup(webpage, 'html.parser')

#Sanity check 
soup

<!DOCTYPE html>

<html lang="en">
<!--Page Generated at Fri Apr 22 02:42:03 2022--><!--Page 2155 Template Prod generated at 4/7/2022 4:00:32 PM-->
<head>
<meta charset="utf-8"/>
<link href="https://layout.eurosport.com/c/v8_5/global.css" rel="stylesheet" type="text/css"/>
<link href="https://layout.eurosport.com/c/v8_5/cycling-realtime-map.css" rel="stylesheet" type="text/css"/><link href="https://layout.eurosport.com/c/v8_5/standing.css" rel="stylesheet" type="text/css"/><link href="https://layout.eurosport.com/c/v8_5/live_bis.css" rel="stylesheet" type="text/css"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<title>Road race - Men live - 17 April 2022 - Eurosport</title><meta content="Watch the Paris - Roubaix event: Road race - Men live on Eurosport. Scores, stats and comments in real time." name="description"/><link href="https://www.eurosport.com/cycling/paris-roubaix/2022/live-road-race-men_mtc1323920/live.shtml" h

In [34]:
# table = soup.find_all('div', attrs={"class":"standing-table"})
soup_head = soup.body
soup_head

# # Let's find the data from the table :
# table = soup.find('table', attrs={'class': 'tableSorter2'})
# # table
# results = table.find_all('tr')
# print('Number of results', len(results))
# print("Table", table)


<body class="cols-2 live withsubnav withsdnav theme-black">
<!--googleoff: all-->
<div id="wrapper">
<section data-analytics-id="header" id="top">
<header id="header">
<div class="header_v8_5"></div>
</header>
<nav id="nav">
<div class="nav_v8_5 sd_navigation"><ul class="sd_nav" data-analytics-id="primary-navigation"><li class="sd_nav-item logo"><a class="sd_nav-item-link header-logo" href="https://www.eurosport.com"></a></li><li class="sd_nav-item sd_navigation_link"><a class="sd_nav-item-link" href="https://video.eurosport.com">Video</a></li><li class="sd_nav-item sd_navigation_link"><a class="sd_nav-item-link" href="/score-center.shtml">Results</a></li><li class="sd_nav-item __popular-sport"><a class="sd_nav-item-link" href="/football/">Football</a></li><li class="sd_nav-item __popular-sport highlighted"><a class="sd_nav-item-link" href="/cycling/">Cycling</a></li><li class="sd_nav-item __popular-sport"><a class="sd_nav-item-link" href="/snooker/">Snooker</a></li><li class="sd_nav-i

### For the next part of the homework you will be scraping data from the following URL

# US FDA web page
https://www.accessdata.fda.gov/scripts/cder/daf/

You are going to go to the FDA web page and search for Fentanyl.  It should return a list of all drugs that contain fentanyl.  The table that comes back is just a simple list of the drug names.  If you click on the drug name, it expands the row to show Drug Name, Link, NDA number, Dosage Form, Marketing Status, and Manufacturer.  You need to scrape this web page and print those values seperated by commas.

### POST forms
POST forms do not put the data in the URL, the data is submitted via javascript.  You have to pretend to fill out the form and submit it.  Grabbing the parameters
First we need to find out what parameters we’re going to hunt down. To do this, first make your way to the form, then get prepared.

1) In Chrome, more tools > Developer Tools 2) Click the Network tab 3) Fill the form out, and submit it 4) Scroll up to the top of the Network pane, select the segment of the URL you’re at (I’m at tempai.cfm) 5) Click it 6) Select Headers on the right 7) Scroll down until you see Form Data

### 1)  Part 1, enter the search term fentanyl and submit

### 2) Pull the data from the website, check the status code to make sure it was successfull and create a BeautifulSoup object and parse the data.  Print the soup object below:

### 3) import selenium webdriver and BeautifulSoup

Get your webdriver and get the page.  Submit the search term and click the search button.

### 4) run the selenium page_source through beautiful soup using the html parser

### 5) Grab all the rows from the table using find_all.  Note that some of the data is in a 'tr' header and some is in 'li'.

Parse the data and print the results

Note the link on the page is a relative link.  You must join that link with 'https://www.accessdata.fda.gov' to make it a link you can click on.
Example:  https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=022510

### The output should look like this (except print all rows)
Drug Name,Link,NDA number,Dosage Form,Marketing Status,Manufacturer

ABSTRAL , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=022510 , NDA   #022510 , TABLET;SUBLINGUAL , Discontinued , SENTYNL THERAPS INC

ACTIQ , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=020747 , NDA   #020747 , TROCHE/LOZENGE;TRANSMUCOSAL , Prescription , CEPHALON

DURAGESIC-100 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS

DURAGESIC-12 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS

DURAGESIC-25 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS


### 10 bonus points for following the link and getting the strength information from that page.  Print the strength information out along with everything you were getting above.